In [1]:
import keras

Using Theano backend.


## 9장. 케라스2 응용
실제 문제에 인공지능을 활용할 때 생기는 문제들을 효율적으로 처리하는 케라스의 응용 기능을 알아봅시다.

### 9.3 실전에서 유용한 케라스 고급 기능
케라스의 유용한 고급 기능을 다룹니다.

In [2]:
# set to use CPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import keras
keras.__version__

'2.0.6'

### 9.3.1 Lambda 계층
#### 9.3.1.1 파이썬 lambda를 함께 활용

In [3]:
from keras.layers import Lambda, Input, Concatenate
from keras.models import Model
import numpy as np

In [7]:
def Lambda_with_lambda():
    x = Input((2,))
    y = Lambda(lambda x: x**2+2*x+1)(x)
    m = Model(x, y)

    yp = m.predict_on_batch(np.array([[1,2],[3,4]]))
    print(yp)
    
Lambda_with_lambda()

/home/sjkim/anaconda3/envs/keras206/lib/python3.6/site-packages/keras/layers/core.py:629: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 2)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


[[ 4.  9.]
 [16. 25.]]


In [9]:
from keras import backend as K
def Backend_for_Lambda():
    def kproc_concat(x):    
        m = K.mean(x, axis=1, keepdims=True)
        d1 = K.abs(x - m)
        d2 = K.square(x - m)
        return K.concatenate([x, d1, d2], axis=1)

    def kshape_concat(input_shape):
        output_shape = list(input_shape)
        output_shape[1] *= 3
        return tuple(output_shape)

    x = Input((3,))
    y = Lambda(kproc_concat, kshape_concat)(x)
    m = Model(x, y)

    yp = m.predict_on_batch(np.array([[1, 2, 3], [3, 4, 8]]))
    print(yp)
    
Backend_for_Lambda()

[[1. 2. 3. 1. 0. 1. 1. 0. 1.]
 [3. 4. 8. 2. 1. 3. 4. 1. 9.]]


In [11]:
import tensorflow as tf
def TF_for_Lamda():
    def kproc_concat(x):    
        m = tf.reduce_mean(x, axis=1, keep_dims=True)
        d1 = tf.abs(x - m)
        d2 = tf.square(x - m)
        return tf.concat([x, d1, d2], axis=1)

    def kshape_concat(input_shape):
        output_shape = list(input_shape)
        output_shape[1] *= 3
        return tuple(output_shape)

    x = Input((3,))
    y = Lambda(kproc_concat, kshape_concat)(x)
    m = Model(x, y)

    yp = m.predict_on_batch(np.array([[1, 2, 3], [3, 4, 8]]))
    print(yp)
    
TF_for_Lamda()

ModuleNotFoundError: No module named 'tensorflow'

In [8]:
def No_Lambda_with_keras2():
    x = Input((2,))
    y = x**2+2*x+1
    m = Model(x, y)

    yp = m.predict_on_batch(np.array([[1,2],[3,4]]))
    print(yp)
    
No_Lambda_with_keras2()

TypeError: Output tensors to a Model must be Keras tensors. Found: Elemwise{add,no_inplace}.0

---
### 9.2.3 전체 코드

In [34]:
from keras.layers import Lambda, Input
from keras.models import Model

def Lambda_with_lambda():
    x = Input((1,))
    y = Lambda(lambda x: x + 1)(x)
    m = Model(x, y)

    yp = m.predict_on_batch(np.array([1, 2, 3]))
    print(yp)


def Lambda_function():
    def kproc(x):
        return x ** 2 + 2 * x + 1

    def kshape(input_shape):
        return input_shape

    x = Input((1,))
    y = Lambda(kproc, kshape)(x)
    m = Model(x, y)

    yp = m.predict_on_batch(np.array([1, 2, 3]))
    print(yp)

    
from keras import backend as K
def Backend_for_Lambda():
    def kproc_concat(x):    
        m = K.mean(x, axis=1, keepdims=True)
        d1 = K.abs(x - m)
        d2 = K.square(x - m)
        return K.concatenate([x, d1, d2], axis=1)

    def kshape_concat(input_shape):
        output_shape = list(input_shape)
        output_shape[1] *= 3
        return tuple(output_shape)

    x = Input((3,))
    y = Lambda(kproc_concat, kshape_concat)(x)
    m = Model(x, y)

    yp = m.predict_on_batch(np.array([[1, 2, 3], [3, 4, 8]]))
    print(yp)

    
import tensorflow as tf
def TF_for_Lamda():
    def kproc_concat(x):    
        m = tf.reduce_mean(x, axis=1, keepdims=True)
        d1 = tf.abs(x - m)
        d2 = tf.square(x - m)
        return tf.concat([x, d1, d2], axis=1)

    def kshape_concat(input_shape):
        output_shape = list(input_shape)
        output_shape[1] *= 3
        return tuple(output_shape)

    x = Input((3,))
    y = Lambda(kproc_concat, kshape_concat)(x)
    m = Model(x, y)

    yp = m.predict_on_batch(np.array([[1, 2, 3], [3, 4, 8]]))
    print(yp)


def main():
    print('Lambda with lambda')
    Lambda_with_lambda()

    print('Lambda function')
    Lambda_function()

    print('Backend for Lambda')
    Backend_for_Lambda()

    print('TF for Lambda')
    TF_for_Lamda()

main()

Lambda with lambda
np.array([1,2,3]) + 1:
[[2.]
 [3.]
 [4.]]
Lambda function
np.array([1,2,3]) + 1:
[[ 4.]
 [ 9.]
 [16.]]
Backend for Lambda
[[1. 2. 3. 1. 0. 1. 1. 0. 1.]
 [3. 4. 8. 2. 1. 3. 4. 1. 9.]]
TF for Lambda
[[1. 2. 3. 1. 0. 1. 1. 0. 1.]
 [3. 4. 8. 2. 1. 3. 4. 1. 9.]]
